In [ ]:
from lec_utils import *
def show_merging_animation():
    src = "https://docs.google.com/presentation/d/1HPJ7fiBLNEURsWYiY0qpqPR3qup68Mr0_B34GU99Y8Q/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 865
    height = 509
    display(IFrame(src, width, height))

<div class="alert alert-info" markdown="1">

#### Lecture 6

# Pivoting, Merging, and Transforming

### EECS 398: Practical Data Science, Winter 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/wn25">github.com/practicaldsc/wn25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/69737/discussion/5943734) </small>
    
</div>

### 3blue1bron 🏀

- [3blue1bron](https://www.3blue1bron.com) allows you to upload a PDF and it'll automatically generate a video of LeBron James summarizing the notes.

- Listen up! 🐐

In [ ]:
IFrame(
    src='https://www.3blue1bron.com/share/7c278b09-f703-454c-9e23-31564f38b040',
    width=800,
    height=700
)

### Agenda 📆

- Recap: `groupby`.
- Pivot tables using `pivot_table`.
- Merging 🚗.
- Transforming 🤖.

## Recap: `groupby`

---

### Loading the data 🐧

<center><img src="imgs/lter_penguins.png" width=500></center>

In [ ]:
penguins = pd.read_csv('data/penguins.csv')
penguins

### The `groupby` method

- The `groupby` method helps us answer questions that involve performing some computation separately **for each group**.

- Most commonly, we'll use `groupby`, select column(s) to operate on, and use a built-in aggregation method.

In [ ]:
# The median 'bill_length_mm' of each 'species'.
...

- There are four other special "grouping methods" we learned about last class that allow for advanced behavior, namely `agg`, `filter`, `transform`, and `apply`.<br><small>See **"⭐️ The grouping method cheat sheet"** from last lecture for examples.</small>

In [ ]:
# The most common 'island' per 'species'.
...

In [ ]:
# Keeps the 'species' with at least 100 penguins.
...

### Grouping with multiple columns

- When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.<br><small>In the output below, why are there only 5 rows, rather than $3 \times 3 = 9$ rows, when there are 3 unique `'species'` and 3 unique `'island'`s?</small>

In [ ]:
# Read this as:
species_and_island = (
    penguins.groupby(['species', 'island'])         # for every combination of 'species' and 'island' in the DataFrame,
    [['bill_length_mm', 'bill_depth_mm']].mean()    # calculate the mean 'bill_length_mm' and the mean 'bill_depth_mm'.
)
species_and_island

- **Advice**: When grouping on multiple columns, the result usually has a `MultiIndex`;  use `reset_index` or set `as_index=False` in `groupby` to avoid this.

In [ ]:
# Now, this looks like a regular DataFrame!
...

## Pivot tables using `pivot_table`

---

### Pivot tables: An extension of grouping

- Pivot tables are a compact way to display tables for humans to read.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>sex</th>
      <th>Female</th>
      <th>Male</th>
    </tr>
    <tr>
      <th>species</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Adelie</th>
      <td>3368.84</td>
      <td>4043.49</td>
    </tr>
    <tr>
      <th>Chinstrap</th>
      <td>3527.21</td>
      <td>3938.97</td>
    </tr>
    <tr>
      <th>Gentoo</th>
      <td>4679.74</td>
      <td>5484.84</td>
    </tr>
  </tbody>
</table>


- Notice that each value in the table is the average of `'body_mass_g'` of penguins, for every combination of `'species'` and `'sex'`.

- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

- The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:
<br><br>
```python
        df.pivot_table(index=index_col,
                       columns=columns_col,
                       values=values_col,
                       aggfunc=func)
```

- The resulting DataFrame will have:
    - One row for every unique value in `index_col`.
    - One column for every unique value in `columns_col`.
    - Values determined by applying `func` on values in `values_col`.

- **Example**: Find the average `'body_mass_g'` for every combination of `'species'` and `'sex'`.

In [ ]:
...

In [ ]:
...

### Example: Finding the number of penguins per `'island'` and `'species'`

In [ ]:
penguins

- Suppose we want to find the number of penguins in `penguins` per `'island'` and `'species'`. We can do so _without_ `pivot_table`:

In [ ]:
...

In [ ]:
...

- But the data is arguably easier to interpret when we do use `pivot_table`:

In [ ]:
...

- Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.<br><small>`NaN` stands for "not a number." It is `numpy` and `pandas`' version of a null value (the regular Python null value is `None`). We'll learn more about how to deal with these soon.</small>

- We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [ ]:
...

### Granularity

- Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

In [ ]:
penguins

- What is the **granularity** of the DataFrame below?<br><small>That is, what does each row represent?</small>

In [ ]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".

- Other DataFrame reshaping methods:
    - `melt`: Un-pivots a DataFrame. Very useful in data cleaning. **See the reference slide!**
    - `pivot`: Like `pivot_table`, but doesn't do aggregation.
    - `stack`: Pivots multi-level columns to multi-indices.
    - `unstack`: Pivots multi-indices to columns.

- Google, the documentation, and ChatGPT are your friends!

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### The `melt` method

- The `melt` method is common enough that we'll give it a special mention.

- We'll often encounter pivot tables (esp. from government data), which we call *wide* data.

- The methods we've introduced work better with *long-form* data, or *tidy* data.

- To go from wide to long, `melt`.

<center><img src='imgs/wide-vs-long.svg' width=400></center>

In [ ]:
wide_example = pd.DataFrame({
    'Year': [2001, 2002],
    'Jan': [10, 130],
    'Feb': [20, 200],
    'Mar': [30, 340]
}).set_index('Year')
wide_example

In [ ]:
wide_example.melt(ignore_index=False)

## Merging 🚗

---

In [ ]:
phones = pd.DataFrame().assign(
    Model=['iPhone 16', 'iPhone 16 Pro Max', 'Samsung Galaxy S24 Ultra', 'Pixel 9 Pro'],
    Price=[799, 1199, 1299, 999],
    Screen=[6.1, 6.9, 6.8, 6.3]
)
inventory = pd.DataFrame().assign(
    Handset=['iPhone 16 Pro Max', 'iPhone 16', 'Pixel 9 Pro', 'Pixel 9 Pro', 'iPhone 16', 'iPhone 15'],
    Units=[50, 40, 10, 15, 100, 5],
    Store=['Briarwood', 'Somerset', 'Arbor Hills', '12 Oaks', 'Briarwood', 'Oakland Mall']
)

### Example: Phone sales 📱

In [ ]:
# The DataFrame on the left contains information about phones on the market.
# The DataFrame on the right contains information about the stock I have in my stores.
dfs_side_by_side(phones, inventory)

- **Question**: If I sell all of the phones in my inventory, how much will I make in revenue?

- The information I need to answer the question is spread across multiple DataFrames.

- The solution is to **merge** the two DataFrames together horizontally.<br><small>The SQL term for merge is **join**.</small>

- A merge is appropriate when we have two sources of information **about the same individuals** that is **linked by a common column(s)**.<br>The common column(s) are called the **join key**.<br>

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
combined = ...
combined

In [ ]:
...

### What just happened!? 🤯

In [ ]:
# Click through the presentation that appears.
show_merging_animation()

### The `merge` method

- The `merge` DataFrame method joins two DataFrames by columns or indexes.<br><small>As mentioned before, "merge" is just the `pandas` word for "join."</small>

- When using the `merge` method, the DataFrame before `merge` is the "left" DataFrame, and the DataFrame passed into `merge` is the "right" DataFrame.<br><small>In `phones.merge(inventory)`, `phones` is considered the "left" DataFrame and `inventory` is the "right" DataFrame.<br>The columns from the left DataFrame appear to the left of the columns from right DataFrame.</small>

- By default:
    - If join keys are not specified, all shared columns between the two DataFrames are used.
    - The "type" of join performed is an inner join, which is just one of many types of joins.

### Inner joins

- The default type of join that `merge` performs is an **inner join**, which keeps the **intersection** of the join keys.

<center><img src='imgs/inner-joins.png' width=300></center>

In [ ]:
# The DataFrame on the far right is the merged DataFrame.
dfs_side_by_side(phones, inventory, phones.merge(inventory, left_on='Model', right_on='Handset'))

- Note that `'Samsung Galaxy S24 Ultra'` and `'iPhone 15'` do not appear in the merged DataFrame.

- That's because there is no `'Samsung Galaxy S24 Ultra'` in the right DataFrame (`inventory`), and no `'iPhone 15'` in the left DataFrame (`phones`).

### Other join types

- We can change the type of join performed by changing the `how` argument in `merge`.

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='left')

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='right')

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='outer')

- The website [pandastutor.com](https://pandastutor.com) can help visualize DataFrame operations like these.<br>[**Here's a direct link**](https://pandastutor.com/vis.html#code=%23%20The%20code%20below%20just%20initializes%20the%0A%23%20phones%20and%20inventory%20DataFrames%20as%20we've%20seen%20them%0A%23%20in%20lecture.%0A%23%20---%0Aimport%20pandas%20as%20pd%0Aimport%20io%0A%0Aphones%20%3D%20pd.read_csv%28io.StringIO%28'''%0AModel,Price,Screen%0AiPhone%2016,799,6.1%0AiPhone%2016%20Pro%20Max,1199,6.9%0ASamsung%20Galaxy%20S24%20Ultra,1299,6.8%0APixel%209%20Pro,999,6.3%0A'''%29%29%0A%0Ainventory%20%3D%20pd.read_csv%28io.StringIO%28'''%0AHandset,Units,Store%0AiPhone%2016%20Pro%20Max,50,Briarwood%0AiPhone%2016,40,Somerset%0APixel%209%20Pro,10,Arbor%20Hills%0APixel%209%20Pro,15,12%20Oaks%0AiPhone%2016,100,Briarwood%0AiPhone%2015,5,Oakland%20Mall%0A'''%29%29%0A%0A%23%20---%0A%0A%23%20Below,%20uncomment%20the%20line%20that%20you'd%20like%20to%20visualize!%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'inner'%29%0A%0Aphones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'left'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'right'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'outer'%29&d=2024-09-11&lang=py&v=v1) to this specific example.

<center><img src="imgs/pt-screenshot.png" width=600></center>

### Different join types handle mismatches differently

<center><img src='imgs/all-joins.png' width=400></center>

- **Inner join**: Keep **only** the matching keys (intersection).

- **Outer join**: Keep **all** keys in both DataFrames (union).

- **Left join**: Keep all keys in the left DataFrame, whether or not they are in the right DataFrame.

- **Right join**: Keep all keys in the right DataFrame, whether or not they are in the left DataFrame.<br><small> Note that `a.merge(b, how='left')` contains the same information as `b.merge(a, how='right')`, just in a different order.</small>

### Tip: Set differences

- A set in Python is a data structure containing **unique**, **unordered** elements.

In [ ]:
phones['Model']

In [ ]:
left = set(phones['Model'])
left

In [ ]:
inventory['Handset']

In [ ]:
right = set(inventory['Handset'])
right

- To quickly check _which_ join key values are in the left DataFrame but not the right, or vice versa, create sets out of the join keys and use the `difference` method.

In [ ]:
...

In [ ]:
...

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Notes on the `merge` method

- `merge` is flexible – you can merge using a combination of columns, or the index of the DataFrame.

-  If the two DataFrames have the same column names, `pandas` will add `_x` and `_y` to the duplicated column names to avoid having columns with the same name (change these the `suffixes` argument).

- There is, in fact, a `join` method, but it's actually a wrapper around `merge` with fewer options.

- **As always, the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) is your friend!**

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Lots of `pandas` operations do an implicit outer join!

- `pandas` will almost always try to match up index values using an outer join.

- It won't tell you that it's doing an outer join, it'll just throw `NaN`s in your result!

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3]}, index=['hello', 'eecs398', 'students'])
df2 = pd.DataFrame({'b': [10, 20, 30]}, index=['eecs398', 'is', 'awesome'])
dfs_side_by_side(df1, df2)

In [ ]:
df1['a'] + df2['b']

### Activity setup

In [ ]:
midwest_cities = pd.DataFrame().assign(
    city=['Ann Arbor', 'Detroit', 'Chicago', 'East Lansing'],
    state=['Michigan', 'Michigan', 'Illinois', 'Michigan'],
    today_high_temp=['79', '83', '87', '87']
)
schools = pd.DataFrame().assign(
    name=['University of Michigan', 'University of Chicago', 'Wayne State University', 'Johns Hopkins University', 'UC San Diego', 'Concordia U-Ann Arbor', 'Michigan State University'], 
    city=['Ann Arbor', 'Chicago', 'Detroit', 'Baltimore', 'La Jolla', 'Ann Arbor', 'East Lansing'],
    state=['Michigan', 'Illinois', 'Michigan', 'Maryland', 'California', 'Michigan', 'Michigan'],
    graduation_rate=[0.87, 0.94, 0.78, 0.92, 0.81, 0.83, 0.91]
)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

**Without writing code**, how many rows are in `midwest_cities.merge(schools, on='city')`?

<br>
    
<center>
A. <b>4</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    B. <b>5</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    C. <b>6</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    D. <b>7</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    E. <b>8</b></center>

In [ ]:
dfs_side_by_side(midwest_cities, schools)

In [ ]:
...

### Followup activity

**Without writing code**, how many rows are in `midwest_cities.merge(schools, on='state')`?

In [ ]:
dfs_side_by_side(midwest_cities, schools)

In [ ]:
...

### The butterfly method 🦋

- A common exam-style question is to determine the number of rows that result from merging two DataFrames, <br>especially when **there are duplicate values in the join keys** (the columns being merged).

- In these questions, use the butterfly method, which says:

> Suppose $x_1, x_2, ..., x_n$ are the **overlapping** values between the join keys of the left DataFrame, `L`, and the right DataFrame, `R`.
> Then, the number of rows in an **inner merge** between `L` and `R` is:
<br>
> $$\text{count}_{L}(x_1) \cdot \text{count}_{R}(x_1) + \text{count}_{L}(x_2) \cdot \text{count}_{R}(x_2) + ... + \text{count}_{L}(x_n) \cdot \text{count}_{R}(x_n) \\ = \sum_{i=1}^n \text{count}_L(x_i) \cdot \text{count}_R(x_i)$$

- We used this method to answer the question on the previous slide!<br>It's called the butterly method because when lines are drawn between the overlapping rows, the result resembles a butterfly (see the posted annotated slides).

- The formula above may seem complicated, but it's a direct consequence of the merging animation we saw earlier.

In [ ]:
show_merging_animation()

- You'll get lots of practice with related problems in this week's discussion worksheet.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

What questions do you have?

## Transforming 🤖

---

### Loading the data 🏦

- [LendingClub](https://www.lendingclub.com/) is a platform that allows individuals to borrow money – that is, take on **loans**.

- For the next 1.5 lectures, we will work with data from their platform.<br>Each row of the dataset corresponds to a different loan that the LendingClub approved and paid out.<br><small>The full dataset is over 300 MB, so we've sampled a subset for this lecture.</small>

In [ ]:
loans = pd.read_csv('data/loans.csv')

In [ ]:
# Each time you run this cell, you'll see a different random subset of the DataFrame.
loans.sample(5)

In [ ]:
# When a DataFrame has more columns than you can see in its preview,
# it's a good idea to check the names of all columns.
loans.columns

- Not all of the columns are necessarily interesting, but here are some that might be:
<br><small>FICO scores refer to credit scores.</small>

In [ ]:
# Again, run this a few times to get a sense of the typical values.
loans[['loan_amnt', 'issue_d', 'term', 'int_rate', 'emp_title', 'fico_range_low']].sample(5)

### Transformations

- A **transformation** results from performing some operation on every element in a sequence, e.g. a Series.


- When **cleaning** data to prepare it for analysis, we often need to:
    - Perform type conversions (e.g. changing the string `'$2.99'` to the float `2.99`).
    - Perform unit conversions (e.g. feet to meters).
    - Extract relevant information from strings.

- For example, we can't currently use the `'term'` column to do any calculations, since its values are stored as strings (despite being numerical).

In [ ]:
loans['term']

- Many of the values in `'emp_title'` are stored inconsistently, meaning they mean the same thing, but appear differently. Without further cleaning, this would make it harder to, for example, find the total number of nurses that were given loans.

In [ ]:
(loans['emp_title'] == 'registered nurse').sum() 

In [ ]:
(loans['emp_title'] == 'nurse').sum() 

In [ ]:
(loans['emp_title'] == 'rn').sum() 

### One solution: The `apply` method

- The Series `apply` method allows us to use a function on every element in a Series.

In [ ]:
def clean_term(term_string):
    ...

In [ ]:
loans['term'].apply(clean_term) 

- There is also an `apply` method for DataFrames, in which you can use a function on every row (if you set `axis=1`) or every column (if you set `axis=0`) of a DataFrame.

- There is also an `apply` method for DataFrameGroupBy/SeriesGroupBy objects, as we discovered last class.

### The price of `apply`

- Unfortunately, `apply` runs really slowly – internally, it just runs a `for`-loop.

In [ ]:
%%timeit
loans['term'].apply(clean_term)

In [ ]:
%%timeit
res = []
for term in loans['term']:
    res.append(clean_term(term))

- So, when possible – say, **when applying arithmetic operations** – we should work on Series objects directly and avoid `apply`.

In [ ]:
%%timeit
loans['int_rate'] // 10 * 10 # Rounds down to the nearest multiple of 10.

In [ ]:
%%timeit
loans['int_rate'].apply(lambda y: y // 10 * 10)

- Above, the solution involving `apply` is ~10x slower than the one that uses direct vectorized operations.

### The `.str` accessor

- For string operations, `pandas` provides a convenient `.str` accessor.<br><small>You've seen examples of it in practice already, with `.str.contains`.

- Mental model: the <span style="color:orange">operation that comes after</span> `.str` is used on every element of <span style="color:blue">the Series that comes before</span> `.str`.

<br>
<center><code><span style="color:blue">s</span>.str.<span style="color:orange">operation</span></code></center>

In [ ]:
# Here, we use .split() on every string in loans['term'].
...

In [ ]:
...

- One might think that it's quicker than `apply`, but it's actually even slower.<br>But, we still use it in practice since it allows us to write concise code.

### Creating timestamps ⏱️

- When dealing with values containing dates and times, it's good practice to convert the values to "timestamp" objects.

In [ ]:
# Stored as strings.
loans['issue_d']

- To do so, we use the `pd.to_datetime` function.<br><small>It takes in a date format string; you can see examples of how they work [**here**](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).</small>

In [ ]:
...

### Aside: The `pipe` method🚰

- There are a few steps we've performed to clean up our dataset.
    - Convert loan `'term'`s to integers.
    - Convert loan issue dates, `'issue_d'`s, to timestamps.

- When we manipulate DataFrames, it's best to define individual functions for each step, then use the `pipe` **method** to chain them all together.<br><small>The `pipe` method takes in a function that maps $\texttt{DataFrame} \rightarrow \texttt{anything}$, but typically $\texttt{anything}$ is a $\texttt{DataFrame}$.</small>

In [ ]:
def clean_term_column(df):
    return df.assign(
        term=df['term'].str.split().str[0].astype(int)
    )
def clean_date_column(df):
    return (
        df
        .assign(date=pd.to_datetime(df['issue_d'], format='%b-%Y'))
        .drop(columns=['issue_d'])
    )

In [ ]:
...

In [ ]:
# Same as above, just way harder to read and write.
...

### Working with timestamps

- We often want to adjust the granularity of timestamps to see overall trends, or seasonality.

- To do so, use the `resample` DataFrame method ([documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).<br><small>Think of it like a version of `groupby`, but for timestamps.</small>

In [ ]:
# This shows us the average interest rate given out to loans in every 6 month interval.
...

- We can also do arithmetic with timestamps.

In [ ]:
# Not meaningful in this example, but possible.
loans['date'].diff() 

In [ ]:
# If each loan was for 60 months,
# this is a Series of when they'd end.
# Unfortunately, pd.DateOffset isn't vectorized, so
# if you'd want to use a different month offset for each row
# (like we'd need to, since some loans are 36 months
# and some are 60 months), you'd need to use `.apply`.
loans['date'] + pd.DateOffset(months=60) 

### The `.dt` accessor

- Like with Series of strings, Series of timestamps have a `.dt` accessor for properties of timestamps ([documentation](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors)).

In [ ]:
loans['date'].dt.year

In [ ]:
loans['date'].dt.month

- You'll use this in Homework 3!

### What's next?

- What is "exploratory data analysis" and how do we do it?

- How do we deal with missing, or null, values?

- How do we create data visualizations in Python?